In [1]:
% matplotlib inline

In [177]:
import random
import time
import numpy as np
import networkx as nx
import gzip, json
import dali, dali.core as D, dali.utils as utils
D.config.default_device = "cpu"
from data_utils import load_dataset, example_to_data
from ontology_utils import load_abstract_trees, remove_cycles

In [4]:
import matplotlib.pyplot as plt

In [5]:
def nodes2diagram(nodes, nodecolouring):
    node2index = {}
    for node in nodes:
        index = len(node2index)
        node2index[node] = index
        
    G = nx.DiGraph()
    
    for node in nodes:
        G.add_node(node2index[node])
        for parent in node.parents:
            if parent in node2index:
                G.add_edge(node2index[parent],  node2index[node])
    labels = {}
    for node in nodes:
        labels[node2index[node]] = str(len(node.children))
        
    colors = [0.0] * len(node2index)
        
    
    for idx, colouring in enumerate(nodecolouring):
        for node in colouring.elements:
            if node in node2index:
                colors[node2index[node]] = (idx + 1.0) / len(nodecolouring)
    
    return G, labels, colors

def nodes2png(nodes, nodecolouring, path, cmap=plt.cm.Accent_r):
    G, labels, colors = nodes2diagram(nodes, nodecolouring)
    
    pos=nx.spring_layout(G)
    
    nx.draw_networkx_nodes(G, pos, node_color=cmap(colors))
    
    nx.draw_networkx_edges(G, pos)
    nx.draw_networkx_labels(G, pos, labels, font_size=10)
    
    plt.savefig(path, bbox_inches="tight")
    plt.clf()

In [10]:
onto = load_abstract_trees("/Users/jonathanraiman/Desktop/datasets/full_wiki_export/ontology.txt.gz")

██████████████████████████████████████████████████████████████ 311.5%


In [11]:
print(len(onto[0].lookup_table))
to_process, activated = remove_cycles(
    onto[0].lookup_table,
    2,
    10,
    1000, 99999999
)
print(len(onto[0].lookup_table))

5630591
4695941
826673
325349
158311
87889
53575
37591
28422
23873
21735
20889
20474
20287
20184
20145
20135
20133
20132
1490 cycles found of size 2 (0.54s)
weird merge (260 => 3215)
weird merge (99 => 1303)
weird merge (637 => 11450)
weird merge (519 => 3932)
weird merge (78 => 1432)
weird merge (1204 => 6910)
weird merge (156 => 2268)
weird merge (873 => 4109)
Merging cycles: 20132 nodes converted into 19080 nodes (search: 0.54s, cycle merge: 0.01s, node merge: 0.27s)
19080
18231
16933
15731
14860
14312
13932
13745
13658
13589
13551
13538
13536
79 cycles found of size 2 (0.07s)
Merging cycles: 13536 nodes converted into 13496 nodes (search: 0.07s, cycle merge: 0.00s, node merge: 0.02s)
13496
13425
13355
13310
13288
13278
13276
13275
13274
13273
34 cycles found of size 2 (0.07s)
Merging cycles: 13273 nodes converted into 13248 nodes (search: 0.07s, cycle merge: 0.00s, node merge: 0.01s)
13248
13242
13238
13235
4 cycles found of size 2 (0.07s)
Merging cycles: 13235 nodes converted into

In [166]:
def collect_all_ancestors(node, ancestors=None):
    if ancestors is None:
        ancestors = []
    if node.history is None:
        ancestors.append(node)
    else:
        for ancestor in node.history:
            collect_all_ancestors(ancestor, ancestors)
    return ancestors
    

In [167]:
ancestors = collect_all_ancestors(a)

In [390]:
with open("ancestors.txt", "wt") as f:
    for ancestor in ancestors:
        f.write(ancestor.name + "\n")
    

In [323]:
from IPython.display import clear_output

def compute_distance_matrix(elements, distance):
    dists = np.zeros((len(elements), len(elements)))
    total = (len(elements) * (len(elements) + 1)) / 2.0
    done = 0
    for i in range(len(elements)):
        for j in range(i + 1, len(elements)):
            dists[i,j] = distance(elements[i], elements[j])
            dists[j,i] = dists[i,j]
            done += 1
            if done % 1000 == 0:
                clear_output(wait=True)
                print("%.3f%% done" % (100.0 * done / total,), flush=True)
        dists[i,i] = 1.0
    return dists

def form_clusters(x, nodes):
    clusters = []
    for i in range(x.max() + 1):
        clusters.append(
            [nodes[idx] for idx in np.where(x == i)[0]]
        )
    return clusters   

In [324]:
big_ancestors = [ancestor for ancestor in ancestors if len(ancestor.children) > 20]
print(len(big_ancestors), flush=True)

dists = compute_distance_matrix(
    big_ancestors,
    lambda x, y: len(set(x.children) & set(y.children)) / (len(set(x.children + y.children)))
)

99.942% done


In [325]:
import sklearn.cluster

prev_memberships = None
cluster_sizes = [30, 40, 41, 42, 45, 50]
for idx, cluster_size in enumerate(cluster_sizes):
    memberships = set()

    spectral = sklearn.cluster.SpectralClustering(n_clusters=n, affinity="precomputed")
    x = spectral.fit_predict(dists)
    
    for i in range(cluster_size):
        memberships.add(tuple(np.where(x == i)[0].tolist()))
    
    print("%d: num empty = %d" % (cluster_size, sum(len(np.where(x == i)[0]) == 0 for i in range(cluster_size))))
    if prev_memberships is None:
        prev_memberships = memberships
    else:
        print("Overlap between %d and %d = %d" % (
                cluster_sizes[idx - 1],
                cluster_sizes[idx],
                len(memberships & prev_memberships)
            )
        )
        prev_memberships = memberships
        
    if idx == len(cluster_sizes) - 1:
        for i in range(cluster_size):
            if len(np.where(x == i)[0]) == 0:
                continue
            print("Cluster %d (%d)" % (i, len( np.where(x == i)[0])))
            for j in np.where(x == i)[0][:10]:
                print(" " * 4 + big_ancestors[j].name)
            if len(np.where(x == i)[0]) > 10:
                print(" " * 4 + "...")
            print("")
    

20: num empty = 0
30: num empty = 0
Overlap between 20 and 30 = 11
40: num empty = 0
Overlap between 30 and 40 = 23
41: num empty = 0
Overlap between 40 and 41 = 25
42: num empty = 1
Overlap between 41 and 42 = 27
45: num empty = 4
Overlap between 42 and 45 = 32
50: num empty = 9
Overlap between 45 and 50 = 31
Cluster 0 (65)
    Category:Physical cosmology
    Category:Large-scale structure of the cosmos
    Category:Extragalactic astronomy
    Category:Virgo Supercluster
    Category:Galaxy superclusters
    Category:Galaxy clusters
    Category:Hydrology and urban planning
    Category:Water management
    Category:Water streams
    Category:Limnology
    ...

Cluster 1 (16)
    Category:Conflicts in 1980
    Category:Conflicts in 1982
    Category:Conflicts in 1981
    Category:Conflicts in 1989
    Category:Conflicts in 1984
    Category:Conflicts in 1983
    Category:Conflicts in 1986
    Category:Conflicts in 1985
    Category:Conflicts in 1987
    Category:Conflicts in 1988
    

/usr/local/lib/python3.4/site-packages/sklearn/manifold/spectral_embedding_.py:217: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


In [354]:
def obtain_improved_ancestry(cluster_assignments, elements, voting_scheme="exclusivity"):
    
    clustered_ancestors = form_clusters(cluster_assignments, elements)
    scores = []
    most_common_parent = []
    
    for cidx, cluster in enumerate(clustered_ancestors):
        parental_score = {}
        all_parents = set([par for node in cluster for par in node.parents])
        for cjdx, other_cluster in enumerate(clustered_ancestors):
            if cidx != cjdx:
                pmi = (len(set(other_cluster) & all_parents) / (len(other_cluster) * len(all_parents)))
                parental_score[cjdx] = pmi
        scores.append(parental_score)

        common_parents = {}
        if voting_scheme == "exclusivity":
            max_score = sum(1.0 / len(node.parents) for node in cluster)
            for parent in all_parents:
                vote = 0.0
                common = 0
                for node in cluster:
                    if parent in node.parents:
                        common += 1
                        vote += 1.0 / (len(node.parents))
                common_parents[parent] = (vote / max_score, common == len(cluster))
        elif voting_scheme == "pmi":
            for node in cluster:
                for par in node.parents:
                    if par not in common_parents:
                        common_parents[par] = (
                            len(overlap) / (len(par.children) * len(cluster)),
                            len(overlap) == len(cluster)
                        )
        else:
            raise ValueError("unrecognized voting scheme %r, choices are pmi or exclusivity" % (voting_scheme,))

        most_common_parent.append(max(common_parents.items(), key=lambda x : x[1][0]))
    return clustered_ancestors, scores, most_common_parent

In [355]:
clustered_ancestors, scores, most_common_parent = obtain_improved_ancestry(x, big_ancestors, "exclusivity")

In [382]:
def produce_ancestry_plan(clustered_ancestors, scores, most_common_parent, unassigned):
    marked_as_parent = [0] * len(clustered_ancestors)
    
    assigned = []
    remainder = []
    
    cluster_children = [set([child for c in cluster for child in c.children]) for cluster in clustered_ancestors]
    
    distance = lambda x, y: len(x & y) / (len(x | y))
    
    new_clusters = [cluster[:] for cluster in clustered_ancestors]
    
    not_assignable = []
    
    for el_idx, el in enumerate(unassigned):
        if len(el.parents) < 3:
            found = None
            for parent in el.parents:
                for cluster in new_clusters:
                    if parent in cluster:
                        found = cluster
                        break
                if found is not None:
                    break
            if found is not None:
                found.append(el)
                continue
        
        dists = []
        y_children = set(el.children)
        for children_set in cluster_children:
            dists.append(
                distance(children_set, y_children)
            )
            
        best_cluster = np.argmax(dists)
        if dists[best_cluster] > 0:
            new_clusters[best_cluster].append(el)
        else:
            not_assignable.append(el)
            
        if el_idx % 10 ==0:
            clear_output(wait=True)
            print("%.3f%% done" % (100.0 * el_idx / len(unassigned),))
        
    
    
    for cidx, (score, (parent, (parent_pmi, parent_full)), cluster) in enumerate(zip(scores, most_common_parent, new_clusters)):
        par = max(score.items(), key=lambda x : x[1])
        
        if parent_pmi == 1.0 and parent_full:
            assigned.append((cluster, cidx, parent))
        else:
            if par[1] > 1e-3:
                print("\n".join([c.name for c in cluster[:2]]))
                print("< ", par[1])
                print("\n".join([c.name for c in new_clusters[par[0]][:2]]))
                print("")
                assigned.append((cluster, cidx, par[0]))
            else:
                remainder.append((cluster, cidx))
            
    print("Assigned %d, remaining %d, unasignables %d" % (
            sum(len(cluster) for cluster, _, _ in assigned),
            sum(len(cluster) for cluster, _ in remainder),
            len(not_assignable)
        )
    )
    return not_assignable

In [383]:
not_assignable = produce_ancestry_plan(
    clustered_ancestors,
    scores,
    most_common_parent,
    [ancestor for ancestor in ancestors if len(ancestor.children) <= 20]
)

99.200% done
Category:Physical cosmology
Category:Large-scale structure of the cosmos
<  0.004201680672268907
Category:Planetary systems
Category:G-type main-sequence stars

Category:Jihadist groups
Category:Organizations designated as terrorist by the United States government
<  0.001893939393939394
Category:Constitutional monarchies
Category:French-speaking countries and territories

Category:History of Costa Rica
Category:Marginal seas of the Pacific Ocean
<  0.0012642225031605564
Category:Military disbanding and disarmament
Category:United Kingdom and the Commonwealth of Nations

Category:New Zealand seafloor (oceanography)
Category:Zealandia (continent)
<  0.01020408163265306
Category:Military disbanding and disarmament
Category:United Kingdom and the Commonwealth of Nations

Category:South African people of Dutch descent
Category:Afrikaner people
<  0.0625
Category:South African people by ethnic or national origin
Category:South African people of European descent

Category:Langua

In [388]:
not_assignable[0].name

'Category:People by religion and century'

In [356]:
marked_as_parent = [0] * len(clustered_ancestors)

for cidx, (score, (parent, (parent_pmi, parent_full)), cluster) in enumerate(zip(scores, most_common_parent, clustered_ancestors)):
    par = max(score.items(), key=lambda x : x[1])
    if par[1] > parent_pmi and par[1] > 1e-3:
        print("Cluster %d < %d" % (cidx, par[0]))
        for node in cluster[:2]:
            print(" " * 4 + node.name)
        print(" -> (%r)" % (par[1]))
        for node in clustered_ancestors[par[0]][:2]:
            print(" " * 4 + node.name)
        marked_as_parent[par[0]] += 1
    else:
        if parent_pmi > 1e-3:
            print("Cluster %d < ? (%s ~ %r %r)" % (cidx, parent.name[:50], parent_pmi, parent_full))
        else:
            print("Cluster %d < ?" % (cidx,))
        for node in cluster[:2]:
            print(" " * 4 + node.name)

Cluster 0 < ? (join(Category:Working conditions,Category:Electron ~ 0.09213323883770377 False)
    Category:Physical cosmology
    Category:Large-scale structure of the cosmos
Cluster 1 < ? (Category:20th-century conflicts by year ~ 1.0 True)
    Category:Conflicts in 1980
    Category:Conflicts in 1982
Cluster 2 < ? (join(Category:Interior design,Category:1st millenn ~ 0.30769230769230765 False)
    Category:Constitutional monarchies
    Category:French-speaking countries and territories
Cluster 3 < ? (join(join(Category:History of the Islamic State of ~ 1.0 True)
    Category:Wars involving Australia
    Category:Global conflicts
Cluster 4 < ? (Category:Rebel militia groups ~ 1.0 True)
    Category:Rebel militia groups in Africa
    Category:Rebel groups by country
Cluster 5 < ? (join(Category:Transport safety,Category:Ancient Eu ~ 0.1746482777739275 False)
    Category:Jihadist groups
    Category:Organizations designated as terrorist by the United States government
Cluster 6 < ? (j

In [357]:
marked_as_parent

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

### Conclusion

40 clusters appears to be sweet spot. Can now recurse on this function to fill in the remainder. Moreover, spectral clustering naturally stops adding clusters -- inherently regularized process.

In [164]:
def node2name(node, max_depth=9999999, label=None):
    if node.history is not None:
        if max_depth == 0:
            return "(...)"
        else:
            return attach_to_history(node.history, max_depth-1, label=label)
    else:
        out = "("
        if label is not None:
            out += str(label) + " "
        else:
            out = out + " "
            
        out += (node.name
                .replace("(", "[")
                .replace(")", "]")
                .replace("Category:", "")) + ")"
        return out
    
import numpy as np

def attach_to_history(nodes, max_depth, label=None):
    nodes = list(nodes)
    children = set(child for node in nodes for child in node.children)
    
    mutual_info = []
    
    for mi_node in nodes:
        other_overlap = set(child for node in nodes for child in node.children if node is not mi_node)
        pmi = len(
                set(mi_node.children) &
                other_overlap
            )
        pmi = pmi / (len(mi_node.children) * len(other_overlap))
        mutual_info.append(pmi)
    
    
    norm_info = np.array(mutual_info)
    norm_info = (norm_info - norm_info.min())
    if norm_info.max() > 0:
        norm_info = norm_info / norm_info.max()
    else:
        norm_info = np.ones_like(norm_info) / len(norm_info)
    norm_info = np.round(norm_info * 4).astype(np.int64)
    
    
    #if len(nodes) > 3:
    #    nodes = list(nodes)[:3]
    out = ""
    #if len(nodes) > 1:
    #    out += "("
    out += "("
    if label is not None:
        out += str(label) + " "
    else:
        out += " "
    out += " ".join([node2name(node, max_depth, norm_info[idx]) for idx, node in enumerate(nodes)]) + ")"
    
    #if len(nodes) > 1:
    #    out += ")"
    return out

In [165]:
node2name(a, 10)

'( (2 (2 Pisces–Cetus Supercluster Complex) (2 (1 (1 Physical cosmology) (3 Large-scale structure of the cosmos) (2 Extragalactic astronomy) (4 Virgo Supercluster) (0 Galaxy superclusters) (1 (2 Local Interstellar Cloud) (0 Galaxy clusters) (1 Milky Way) (0 (0 2nd millennium in the United States) (1 (0 (2 Administrative regions of Greece) (1 (2 (...) (0 Establishments by millennium) (4 Events by year) (1 Early Christianity)) (2 Jewish religious movements)) (0 Categories by administrative region of Greece) (4 Traditional geographic divisions of Greece) (1 Central Greece)) (0 Ancient Central Greece) (4 Local government in Greece)) (0 Millennia in the United States) (4 History of the United States [1945–64]) (2 20th century in the United States) (0 Millennia in North America)) (4 Milky Way Subgroup) (1 Spiral galaxies) (0 Gould Belt) (1 Vortices) (1 Galaxies) (1 Barred spiral galaxies) (2 Local Bubble) (0 Orion–Cygnus Arm) (2 Barred galaxies) (2 Local Group))) (4 Laniakea Supercluster) (2

In [18]:
joined = [(key, onto[0].lookup_table[key]) for key in onto[0].lookup_table.keys() if key.startswith("join(")]

In [20]:
joined_s = sorted([(j.count("join("), node) for j, node in joined], key=lambda x : x[0])[::-1]

In [89]:
a = onto_backup[0].lookup_table[max(joined, key=lambda x : x.count("join("))]

In [93]:
a.parents

[<Branch name="Category:Concepts in physics", num_children=403, num_parents=0>,
 <Branch name="Category:Analytic philosophy", num_children=59, num_parents=0>,
 <Branch name="Category:Lexicology", num_children=30, num_parents=0>,
 <Branch name="Category:Skills", num_children=42, num_parents=0>,
 <Branch name="Category:Main topic classifications", num_children=16, num_parents=0>,
 <Branch name="Category:Communication", num_children=261, num_parents=1>,
 <Branch name="Category:Words and phrases", num_children=8, num_parents=1>,
 <Branch name="Category:Rules", num_children=44, num_parents=1>,
 <Branch name="Category:Terminology", num_children=102, num_parents=2>,
 <Branch name="Category:Biota", num_children=10, num_parents=2>,
 <Branch name="Category:Language", num_children=89, num_parents=2>,
 <Branch name="Category:Taxonomy", num_children=101, num_parents=2>,
 <Branch name="Category:Biological classification", num_children=44, num_parents=0>,
 <Branch name="Category:Writing", num_childre

In [ ]:
dataset = load_dataset("/Users/jonathanraiman/Desktop/datasets/mammals_onto/train.tsv.gz")

In [ ]:
onto = load_abstract_trees("/Users/jonathanraiman/Desktop/datasets/sports_ontology.txt.gz")

In [48]:
onto = load_abstract_trees("/Users/jonathanraiman/Desktop/datasets/mammals_onto/ontology.txt.gz")

█████████████████████████████████████████████████████████ 288.3%


In [49]:
def deepcopy_tree(lookup_table):
    copy_table = {}
    
    for key in lookup_table:
        alternate = OntologyBranch(key)
        copy_table[key] = alternate
        
    for key, value in lookup_table.items():
        alternate = copy_table[key]
        alternate.parents = [copy_table[parent.name]
                             for parent in value.parents]
        alternate.children = [copy_table[child.name]
                             for child in value.children]
        
        if value.lookup_table is not None:
            alternate.lookup_table = copy_table
            
    return copy_table

In [12]:
print("\n".join([key for key in onto[0].lookup_table.keys() if key.startswith("join(")]))

join(Category:Asante Kotoko F.C. players,Category:Asante Kotoko)
join(Category:History of the Ottawa Senators,Category:Ottawa Senators (original))
join(Category:Victoria Beckham,Category:David Beckham)
join(Category:Fiction,join(join(Category:Narratology,Category:Genres),Category:Drama))
join(join(Category:Tibetan Buddhism in India,Category:Indo-Tibetan Buddhism),Category:Tibetan Buddhism)
join(Category:Motorcycle manufacturers of Australia,Category:Australian motorcycles)
join(Category:Swordsmanship,Category:Swords)
join(Category:American Basketball Association (2000–present) venues,Category:Basketball venues in the United States)
join(join(Category:Unreal Engine games,Category:Unreal (series)),Category:Unreal Engine)
join(Category:Texas Chaparrals,Category:Dallas Chaparrals)
join(join(Category:AS Saint Estève players,Category:XIII Catalan players),Category:Catalans Dragons players)
join(Category:Scottish rugby union competitions,Category:Rugby union competitions in Scotland)
join(Cat

In [17]:
subparent = {}

for node in to_process:
    for child in node.children:
        if child not in to_process and child not in activated:
            if child.name in subparent:
                subparent[child.name].append(child)
            else:
                subparent[child.name] = [child]

In [25]:
necessaries = set([el for group in subparent.values() for el in group])

In [26]:
extended_blocking_group = necessaries | to_process

In [36]:
lookup_table = onto[0].lookup_table

In [37]:
merged = merge_cycles(extended_blocking_group, lookup_table)

In [22]:
sum(map(len, subparent.values()))

3035

In [85]:
def find_3cycles(nodes):
    cycle3 = []
    for node in nodes:
        for child in node.children:
            if node in child.children:
                cycle3.append(node)
                break
    return cycle3
    

def find_3cycles(nodes):
    cycle4 = []
    for node in nodes:
        for child in node.children:
            for subchild in child.children:
                if node in subchild.children:
                    cycle4.append(node, (child, subchild, node))
                    break
    return cycle4

In [95]:
cycles = []
node2cycle = {}

for el in to_process:
    f = find_ncycles(el, 3)
    for els in f:
        cycle = Cycle(set(els))
        for key in cycle.elements:
            if key in node2cycle:
                node2cycle[key].elements = node2cycle[key].elements | cycle.elements
            else:
                node2cycle[key] = cycle

In [97]:
len(set(node2cycle.values()))

2

In [54]:
find_4cycles(extended_blocking_group)

[<Branch name="Category:Genetic genealogy", num_children=79, num_parents=2>,
 <Branch name="Category:Gender studies", num_children=175, num_parents=2>,
 <Branch name="Category:Gender equality", num_children=88, num_parents=1>,
 <Branch name="Category:Modern human genetic history", num_children=11, num_parents=2>,
 <Branch name="Category:Recent single origin hypothesis", num_children=40, num_parents=2>,
 <Branch name="Category:Feminism", num_children=124, num_parents=4>]

In [53]:
find_4cycles(to_process)

6

In [27]:
leaves = [val for val in onto[0].lookup_table.values() if len(val.children) == 0]

In [7]:
weights = {}
for leaf in leaves:
    weights[leaf.name] = 1.0

In [8]:
def normalize_weights(root, weights):
    total = compute_weight(root, weights)
    for key in weights.keys():
        weights[key] /= total
    return total

In [9]:
# total = normalize_weights(onto[0], weights)

In [28]:
def compute_weight(leaves, weights):
    visited = set()
    other_weights = {}
    to_process = set(leaves)
    
    prev_new_candidates = None
    
    while len(to_process) > 0:
        print(len(to_process))
        new_candidates = {}
        for candidate in to_process:
            if len(candidate.children) == 0:
                weight = weights[candidate.name]
                other_weights[candidate.name] = weight
                
                for parent in candidate.parents:
                    if parent.name not in visited:
                        new_candidates[parent.name] = parent
                        visited.add(parent.name)
            else:
                if all(child.name in other_weights for child in candidate.children):
                    weight = sum(other_weights[child.name] for child in candidate.children)
                    other_weights[candidate.name] = weight
                    for parent in candidate.parents:
                        if parent.name not in visited:
                            new_candidates[parent.name] = parent
                            visited.add(parent.name)
                else:
                    new_candidates[candidate.name] = candidate
                
                    
        to_process = set(new_candidates.values())
        if prev_new_candidates == new_candidates:
            #prev = len(to_process)
            #new_to_process = merge_cycles(to_process)
            #now = len(new_to_process)
            #print("Merging cycles %d nodes converted into %d nodes" % (
            #    prev, now
            #))
            #if now > 0:
            #    to_process = new_to_process
            #else:
            print("error")
            break
        else:
            prev_new_candidates = new_candidates
    
    return other_weights, to_process

In [30]:
joined = [value for key, value in onto[0].lookup_table.items() if key.startswith("join(")]

In [31]:
a = max(joined, key=lambda node : node.name.count("join("))

In [60]:
list(a.history)[0].name[:100], list(a.history)[1].name[:100]

('join(Category:Pisces–Cetus Supercluster Complex,join(join(Category:Physical cosmology,Category:Large',
 'Category:Galaxy filaments')

In [52]:
david_beckham = onto[0].lookup_table["David Beckham"]
david_beckham.parents

[<Branch name="Category:FIFA 100", num_children=125, num_parents=3>,
 <Branch name="Category:Living people", num_children=716514, num_parents=1>,
 <Branch name="Category:BBC Sports Personality of the Year winners", num_children=59, num_parents=1>,
 <Branch name="Category:English Football Hall of Fame inductees", num_children=140, num_parents=2>,
 <Branch name="Category:FIFA Century Club", num_children=503, num_parents=2>,
 <Branch name="Category:England international footballers", num_children=1216, num_parents=3>,
 <Branch name="Category:The Football League players", num_children=19324, num_parents=3>,
 <Branch name="Category:Serie A players", num_children=4199, num_parents=3>,
 <Branch name="Category:Premier League players", num_children=3561, num_parents=3>,
 <Branch name="Category:Association football midfielders", num_children=19092, num_parents=1>,
 <Branch name="Category:La Liga players", num_children=3863, num_parents=3>,
 <Branch name="Category:Ligue 1 players", num_children=3